# Trabalho final de Sistemas Baseados em Conhecimento
### Tema: Geração de texto com Markov
### Alunos: Caio Lucas da Silva Chacon (20200025769) e Jonas Gabriel Leite de Araújo (20200007608)

### Dado o início do texto em português, gerar automaticamente algo que faça sentido usando cadeias de Markov

In [2]:
from scipy.sparse import dok_matrix
import numpy as np
import re
from random import random 

# Leitura do arquivo e processamento textual para melhor análise

def read_text(filename):
    """"
    Função que lê um arquivo de texto e retorna o conteúdo em minúsculas
    """

    with open(filename, 'r', encoding="utf8") as file:
        content = file.read().lower()

        content = re.sub(r'[\n,\.;:\"\'_\-\t\—]', ' ', content)
        content = re.sub(r'\s+', ' ', content)

    return content

content = read_text('domCasmurro.txt')
print(content)


dom casmurro capítulo primeiro do título uma noite destas vindo da cidade para o engenho novo encontrei no trem da central um rapaz aqui do bairro que eu conheço de vista e de chapéu cumprimentou me sentou se ao pé de mim falou da lua e dos ministros e acabou recitando me versos a viagem era curta e os versos pode ser que não fossem inteiramente maus sucedeu porém que como eu estava cansado fechei os olhos três ou quatro vezes tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso continue disse eu acordando já acabei murmurou ele são muito bonitos vi lhe fazer um gesto para tirá los outra vez do bolso mas não passou do gesto estava amuado no dia seguinte entrou a dizer de mim nomes feios e acabou alcunhando me dom casmurro os vizinhos que não gostam dos meus hábitos reclusos e calados deram curso à alcunha que afinal pegou nem por isso me zanguei contei a anedota aos amigos da cidade e eles por graça chamam me assim alguns em bilhetes dom casmurro domingo vou j

In [3]:
def get_words(corpus):
    """
    Retorna todas as palavras
    """
    corpus_words = corpus.split(' ')
    corpus_words= [word for word in corpus_words if word != '']
    corpus_words # [...'a', 'wyvern', ',', 'two', 'of', 'the', 'thousand'...]
    return corpus_words # 2185920

def get_distinct_words_count(corpus_words):
    """
    Função que retorna a quantidade de palavras distintas e um dicionário com o índice de cada palavra
    """

    distinct_words = list(set(corpus_words))
    word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
    distinct_words_count = len(list(set(corpus_words)))
    return distinct_words_count, word_idx_dict # 32663

In [4]:
corpus_words = get_words(content)
distinct_words = list(set(corpus_words))

distinct_words_count, word_idx_dict = get_distinct_words_count(corpus_words)
#word_idx_dict = distinct_words_count[1]
#distinct_words_count = distinct_words_count[0]
print(distinct_words_count)
print(word_idx_dict)

9129
{'morros': 0, 'refletia': 1, 'considerando': 2, 'poetas': 3, 'ajudar': 4, 'pêndulo': 5, 'jantares': 6, 'cala': 7, 'jeito': 8, 'rosto': 9, 'descuido': 10, 'carola': 11, 'obedecia': 12, 'comeram': 13, 'heitor': 14, 'desejar': 15, 'moça”': 16, 'louvada': 17, 'devia': 18, 'fugisse': 19, 'brevidade': 20, 'cujo': 21, 'boneca': 22, 'integralmente': 23, 'unidas': 24, 'monotonia': 25, 'graça?': 26, 'salas': 27, 'falto': 28, 'narrada': 29, 'médico': 30, 'poupar': 31, 'relembrar': 32, 'nascer': 33, 'suponde': 34, 'violento': 35, 'ríspidas': 36, 'reitor': 37, 'iria': 38, 'dacolá': 39, 'cvii': 40, 'dorso': 41, 'arquearam': 42, 'credor': 43, 'sepultado': 44, 'pitanga': 45, 'leitão': 46, 'história!': 47, 'pigarrear': 48, 'trazia': 49, 'egoísmo': 50, 'fizéssemos': 51, 'sentei': 52, 'escrever': 53, 'fugindo': 54, 'pátria': 55, 'perseguiam': 56, 'voltava': 57, 'feições': 58, 'comprando': 59, 'repreender': 60, 'ninfa': 61, 'comentando': 62, 'abraçados': 63, 'teologia': 64, 'lavava': 65, 'moram': 66,

In [30]:
n_grams = 2 # Numero de palavras anteriores para prever a próxima
# Criação de conjuntos de palavras de tamanho n_grams
sets_of_k_words = [ ' '.join(corpus_words[i:i+n_grams]) for i, _ in enumerate(corpus_words[:-n_grams]) ]
sets_count = len(list(set(sets_of_k_words)))

print(sets_of_k_words[:10])

['dom casmurro capítulo', 'casmurro capítulo primeiro', 'capítulo primeiro do', 'primeiro do título', 'do título uma', 'título uma noite', 'uma noite destas', 'noite destas vindo', 'destas vindo da', 'vindo da cidade']


In [31]:
# Criação de uma matriz esparsa para armazenar a contagem de palavras
next_after_k_words_matrix = dok_matrix((sets_count, distinct_words_count))

# Criação de um dicionário para mapear os conjuntos de palavras para índices
distinct_sets_of_k_words = list(set(sets_of_k_words))
k_words_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_k_words)}

# Preenchimento da matriz com as contagens
for i, word in enumerate(sets_of_k_words[:-n_grams]):
    word_sequence_idx = k_words_idx_dict[word]
    next_word_idx = word_idx_dict[corpus_words[i+n_grams]]
    next_after_k_words_matrix[word_sequence_idx, next_word_idx] +=1
    
print("Quantidade de vezes que os bigramas ocorrem juntos")
print(next_after_k_words_matrix[:10])

Quantidade de vezes que os bigramas ocorrem juntos
  (8, 6473)	1.0
  (7, 177)	1.0
  (2, 8148)	1.0
  (3, 2382)	1.0
  (9, 7893)	1.0
  (5, 5898)	1.0
  (4, 6473)	1.0
  (0, 7181)	1.0
  (6, 3968)	1.0
  (9, 1563)	1.0
  (1, 1563)	1.0


In [32]:
def weighted_choice(objects, weights):
    """ 
    Função que retorna um elemento aleatório da sequência de 'objetos',
    a probabilidade dos objetos é ponderada de acordo com a porcentagem.
    """
    weights = np.array(weights, dtype=np.float64)
    sum_of_weights = weights.sum()
    # standardization:
    np.multiply(weights, 1 / sum_of_weights, weights)
    weights = weights.cumsum()
    x = random()
    for i in range(len(weights)):
        if x < weights[i]:
            return objects[i]

In [33]:
def sample_next_word_after_sequence(word_sequence, alpha = 0):
    """
    Função que retorna a próxima palavra após a sequência de palavras
    """
    next_word_vector = next_after_k_words_matrix[k_words_idx_dict[word_sequence]] + alpha
    likelihoods = next_word_vector/next_word_vector.sum()
    
    return weighted_choice(distinct_words, likelihoods.toarray())
    
def markov_chain(initial_text, chain_length=30):
    """
    Função que gera um texto de acordo com o texto inicial
    """
    current_words = initial_text.split(' ')
    sentence = initial_text

    for _ in range(chain_length):
        sentence+=' '
        next_word = sample_next_word_after_sequence(' '.join(current_words))
        sentence+=next_word
        current_words = current_words[1:]+[next_word]
    return sentence

In [44]:
markov_chain('dom casmurro não', chain_length=50)

'dom casmurro não cuide que o dispenso do teatro amanhã venha e dormirá aqui na cidade dou lhe camarote dou lhe chá dou lhe cama só não lhe dou moça não consultes dicionários casmurro não está aqui no sentido que eles lhe dão mas no que lhe pôs o vulgo de homem calado'

In [36]:
markov_chain('capitu não era', chain_length=50)

'capitu não era menos terna para ele e para mim dávamos as mãos um ao outro e quando não olhávamos para o nosso lado o lado de papai não precisa revirar os olhos assim assim era depois de jantar fui levá lo à porta onde esperamos a passagem de um cavaleiro um dandi'

In [42]:
markov_chain('agora que expliquei', chain_length=50)

'agora que expliquei o título passo a escrever o livro antes disso porém digamos os motivos que me põem a pena na mão sim nero augusto massinissa e tu grande césar que me incitas a fazer os meus comentários agradeço vos o conselho e vou deitar ao papel as reminiscências que me vierem'